<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/all_in_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install exchange_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 4.9 MB/s eta 0:00:00


In [21]:
!pip install finance-datareader

In [22]:
import pandas as pd
import numpy as np
import requests
import datetime
import exchange_calendars as ecals # 개장일만
from io import StringIO
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import nltk
import requests
from bs4 import BeautifulSoup

In [23]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

250일 등락률, 거래대금 90-99: 범인매매

60일 등락률 50-100, 거래대금 10-50 : 조용히 오르는 애들 (내꺼)



In [24]:
class StockList():
  def __init__(self, period=250, increased=[.9, .99], traded=[.9, .99], pre_period=0):
    self.period = period
    self.increased = increased
    self.traded = traded
    self.pre_period = pre_period # 시작기점

    self.price_dic = {}

    self.start, self.today = self.get_date()
    self.df = self.get_stock_df()
    self.filtered_df = self.get_filtered_df()
    self.result_df = self.get_result_df()

  def get_date(self):
    today = datetime.date.today().strftime('%Y%m%d')
    if self.pre_period != 0:
      today = (datetime.date.today() - datetime.timedelta(days=self.pre_period)).strftime('%Y%m%d')
    start = (datetime.date.today() - datetime.timedelta(days=self.period)).strftime('%Y%m%d')

    if XKRX.is_session(today) == False:
      today = XKRX.previous_open(today).strftime('%Y%m%d')
    if XKRX.is_session(start) == False:
      start = XKRX.next_open(start).strftime('%Y%m%d')
    return start, today

  def get_stocks(self, market='STK'):
    data = {
      'mktId': market,
      'strtDd': self.start,
      'endDd': self.today,
      'money': '1',
      'adjStkPrc': '2',
      'adjStkPrc_check': 'Y',
      'share': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
    }
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_key = requests.post(gen_url, data=data)

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    r = requests.post(down_url, data={'code':gen_key.text})
    r.encoding = 'EUC-KR'
    return pd.read_csv(StringIO(r.text))

  def get_stock_df(self):
    return pd.concat([self.get_stocks(), self.get_stocks('KSQ')]).reset_index(drop=True)

  def get_filtered_df(self):
    traded_df = self.df[(self.df['거래대금'] < self.df['거래대금'].quantile(self.traded[1])) & (self.df['거래대금'] > self.df['거래대금'].quantile(self.traded[0]))]
    increased_df = self.df[(self.df['등락률'] > self.df['등락률'].quantile(self.increased[0])) & (self.df['등락률'] < self.df['등락률'].quantile(self.increased[1]))]
    selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))
    return self.df[self.df['종목명'].isin(selected)].sort_values('등락률', ascending=False).head(20)

  def get_sharpe(self, df):
    change = df['Change']+1
    return change.mean()/change.std()

  def get_sortino(self, df):
    change = df['Change']+1
    return change.mean()/(change[change<1]).std()

  def get_position(self, df):
    return df['Close'][-1]/df['Close'].max()

  def get_future_mdd(self, s):
    peak = s.iloc[0]
    max_drawdown = 0
    for price in s:
        if price > peak:
            peak = price
        drawdown = (peak - price) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown

  def get_hashtags(self, code):
    page = 5
    li = []
    for i in range(1, page):
      url = 'https://finance.naver.com/item/news_news.naver?code=' + code + '&page=' + str(i) + '&sm=title_entity_id.basic&clusterId='
      result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
      bs_obj = BeautifulSoup(result.content, "html.parser")
      tds = bs_obj.find_all('td', {'class': 'title'})
      texts = [td.text.strip() for td in tds]
      li.extend([sentence.replace('[', ' ').replace(']', ' ').replace('…', ' ').replace('·', ' ').replace('"', ' ').replace('“', ' ').replace(',', ' ').replace('...', ' ').replace("'", ' ').replace('‘', ' ').replace('’', ' ') for sentence in texts])

    words = [word for sentence in li for word in sentence.split()]
    stop_words = ['%', '에', '전망', '수혜', '추가', '주가', '주식', '특징주', \
                  '코스닥', '코스피', '시총', '전년비', '1분기', '2분기', '3분기', \
                  '4분기', '전년比', '↑', '지난해', '영업익', '대비', '증가', '영업이익',\
                  '규모', '속', '주당', '매출', '/', '소', '전년', '1Q', '2Q', '3Q', \
                  '4Q', '관련', '데이터로', '보는', '증시', '영향', '등']
    words = [word for word in words if word not in stop_words]
    word_frequencies = nltk.FreqDist(words)
    keywords = word_frequencies.most_common(11)
    return ' '.join(['#'+x[0] for x in keywords[1:]])

  def get_price_df(self, codes):
    d = {}
    for i in codes:
      d[i] = fdr.DataReader(str(i), self.start, self.today)
    return d

  def get_result_df(self, hashtags=False):
    self.price_dic = self.get_price_df(self.filtered_df['종목코드'])

    if (self.pre_period != 0):
      future_dic = {}
      for i in self.filtered_df['종목코드']:
        future = (datetime.date.today() + datetime.timedelta(days=self.pre_period)).strftime('%Y%m%d')
        future_dic[i] = fdr.DataReader(str(i), self.today, future)
      self.filtered_df['mdd'] = [round(self.get_future_mdd(future_dic[x]['Close']), 4) for x in future_dic]
      self.filtered_df['Performance'] = [round((future_dic[x]['Close'][-1]/future_dic[x]['Close'][0]), 4) for x in future_dic]
      self.filtered_df['preperiod'] = [self.pre_period for x in future_dic]
    else:
      self.filtered_df['mdd'] = [0 for x in self.price_dic]
      self.filtered_df['Performance'] = [0 for x in self.price_dic]
      self.filtered_df['preperiod'] = [self.pre_period for x in self.price_dic]

    self.filtered_df['Sharpe'] = [self.get_sharpe(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Sortino'] = [self.get_sortino(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Position'] = [self.get_position(self.price_dic[x]) for x in self.price_dic]

    if hashtags == True:
      self.filtered_df['Hashtags'] = [self.get_hashtags(x) for x in self.price_dic.keys()]
    else:
      self.filtered_df['Hashtags'] = [0 for x in self.price_dic.keys()]

    t = self.filtered_df.sort_values('Sharpe', ascending=False)

    scaler = MinMaxScaler(feature_range=(5, 10))

    t['Position'] = scaler.fit_transform(t[['Position']]).round(1)
    t['Sortino'] = scaler.fit_transform(t[['Sortino']]).round(1)
    t['Sharpe'] = scaler.fit_transform(t[['Sharpe']]).round(1)
    t['거래대금'] = scaler.fit_transform(t[['거래대금']]).round(1)
    t['등락률'] = scaler.fit_transform(t[['등락률']]).round(1)
    t['합산'] = t['Position'] + t['Sortino'] + t['Sharpe'] + t['거래대금'] + t['등락률']
    t['합산'] = scaler.fit_transform(t[['합산']]).round(1)

    t = t[['종목명', '종료일 종가', '등락률', '거래대금',	'Sharpe',	'Sortino',	'Position', '합산', 'Performance', 'Hashtags']].reset_index(drop=True).rename({'종료일 종가':'현재가', '등락률':'파워', '거래대금':'관심도', 'Sharpe':'Risk1', 'Sortino':'Risk2', 'Position':'모멘텀'}, axis=1)
    t.index+=1
    t.index.name='순위'

    return t

In [25]:
# k
stocklist = StockList(365, [.98,1], [.99,1]) # 원래 250인데 365로 바꿈
t = stocklist.filtered_df
t = t.sort_values(['거래대금'], ascending=[False]).head(3)

In [ ]:
# k trend

d = {}
d = stocklist.get_price_df(t['종목코드'].values)

merged_df = pd.DataFrame()
num_intervals = 15

num_intervals = 15

for key, df in d.items():
    increased_ratio = df['Close'] / df['Close'].iloc[0]
    interval_size = int((len(increased_ratio) - 1) / (num_intervals - 1))
    row_indices = list(range(0, len(increased_ratio), interval_size))
    row_indices[-1] = len(increased_ratio) - 1
    selected_ratio = increased_ratio.iloc[row_indices]
    merged_df[key] = selected_ratio

merged_df

,086520,247540,003670
Date,,,
2022-08-04,1.000000,1.000000,1.000000
2022-08-30,1.059569,0.875781,1.239852
2022-09-26,1.230465,0.721094,1.225092
2022-10-21,1.435545,0.831250,1.380074
2022-11-15,1.297857,0.910937,1.741697
2022-12-08,1.182619,0.808594,1.505535
2023-01-03,1.146143,0.728125,1.383764
2023-01-30,1.272822,0.807813,1.608856
2023-02-22,2.387799,1.253906,1.616236


In [ ]:
from google.colab import files

merged_df.to_csv('king_trend_df.csv', encoding="utf-8-sig")
files.download('king_trend_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
t = t[['종목명','등락률','거래대금']].reset_index(drop=True)
t.index+=1
t.index.name='순위'
total_traded = t['거래대금'].sum()
t['거래대금'] = (t['거래대금'] / total_traded) * 100
t

,종목명,등락률,거래대금
순위,,,
1,에코프로,1122.10,41.963304
2,에코프로비엠,198.43,32.137681
3,포스코퓨처엠,252.01,25.899015


In [ ]:
from google.colab import files

t.to_csv('king_df.csv', encoding="utf-8-sig")
files.download('king_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Ai
stocklist = StockList()
t = stocklist.get_result_df(hashtags=True).head(15)
t

,종목명,현재가,파워,관심도,Risk1,Risk2,모멘텀,합산,Performance,Hashtags
순위,,,,,,,,,,
1,한화오션,46500,5.5,5.8,10.0,8.2,9.9,8.9,0,#잠수함 #국산화 #첫 #음향장비 #수주 #핵심 #성공 #개발 #호위함 #후
2,이오테크닉스,165200,5.3,5.0,9.2,10.0,9.9,8.9,0,#310억 #0.8% #↓ #JYP엔터 #기관 #외국인 #순 #윈텍 #엑시트 #나선다
3,HPSP,36700,5.5,6.8,8.1,7.2,10.0,8.3,0,#한미반도체 #반도체 #실적 #결정 #100억 #자사주 #투자 #기관 #외국인 #업황
4,칩스앤미디어,37850,5.5,5.0,8.0,6.8,9.0,7.3,0,#16억 #취득 #신탁계약 #감소 #AI #20억 #자사주 #확대 #2.9% #45...
5,ISC,92700,6.3,5.6,7.8,6.5,9.1,7.6,0,#SKC #반도체 #인수 #테스트 #솔루션 #소켓 #기업 #아이에스시(ISC) #아...
6,한미반도체,46950,9.6,10.0,7.5,6.7,9.2,10.0,0,#반도체 #HBM #AI #장비 #본더팩토리 #10대 #수요 #실적 #세계 #오픈
7,미래컴퍼니,46450,5.2,5.1,7.0,7.5,9.5,7.3,0,#공급 #디스플레이 #공급계약 #계약 #장비 #제조장비 #레보아이 #체결 #258억...
8,삼아알미늄,105900,8.7,6.0,6.9,6.4,8.4,7.9,0,#알루미늄박 #6951억 #LG에너지솔루션에 #계약 #2차전지 #공시 #LG엔솔 #...
9,레이크머티리얼즈,18970,10.0,9.3,6.7,5.9,8.1,9.1,0,#83억 #시설투자 #94억 #기관 #외국인 #7.7%↑ #전고체 #핵심 #생산 #...


In [ ]:
t.to_csv('ai_df.csv', encoding="utf-8-sig")
files.download('ai_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# leading
# 2주/4주 간격 업데이트
exclude_list = t.head(3)['종목명'].values
leading_df = t[~t['종목명'].isin(exclude_list)]
data = {
    '날짜': ['2022', '2022', '2022'],
    '종목명': leading_df.sort_values('합산', ascending=False).head(3)['종목명'].values,
    '매수가': [0, 0, 0],
    '목표가': [0, 0, 0],
    '손절가': [0, 0, 0],
    '목표수익률': [0, 0, 0],
}
leading_df = pd.DataFrame(data)
leading_df

,날짜,종목명,매수가,목표가,손절가,목표수익률
0,,한미반도체,0,0,0,0
1,,레이크머티리얼즈,0,0,0,0
2,,삼아알미늄,0,0,0,0


In [ ]:
leading_df.to_csv('leading_df.csv', encoding="utf-8-sig")
files.download('leading_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# performance
all_df = []
for p in [20, 60, 120]:
  stocklist = StockList(period=250, increased=[.9, .99], traded=[.9, .99], pre_period=p)
  t = stocklist.filtered_df
  t = t[['종목명', '종료일 종가', 'mdd', 'Performance', 'preperiod']].reset_index(drop=True)
  t.index+=1
  t.index.name='순위'
  all_df.append(t)
t = pd.concat(all_df)
t = t.groupby("preperiod").apply(lambda x: x.sort_values("순위", ascending=True).head(10))
t = t.sort_values('Performance', ascending=False)
t = t.drop_duplicates(subset=['종목명'], keep='first')
t = t.sample(frac=1).reset_index(drop=True)
t.index.name='순위'

In [ ]:
t

,종목명,종료일 종가,mdd,Performance,preperiod
순위,,,,,
0,이브이첨단소재,6600,0.2324,0.8152,20
1,지오릿에너지,13400,0.4266,0.6425,60
2,브리지텍,11100,0.4045,0.7252,120
3,덕양산업,8770,0.2611,0.7400,20
4,DB,2100,0.1862,0.9410,60
5,티로보틱스,17180,0.1634,1.6647,60
6,삼천당제약,84300,0.4903,0.7948,120
7,셀바스헬스케어,6290,0.1655,1.6057,20
8,삼부토건,4795,0.3014,0.7508,20


In [ ]:
t.to_csv('performance_df.csv', encoding="utf-8-sig")
files.download('performance_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# sentiments

!pip install newsapi-python
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
from newsapi import NewsApiClient
import datetime
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from wordcloud import WordCloud
import re

api_key = '6147ad35d10843b2949edc41cd955155'
api = NewsApiClient(api_key=api_key)

In [ ]:
# business general

def get_headline_sentiment(category='business'):
    df = api.get_top_headlines(country='us', category=category, page_size=100)
    df = pd.DataFrame(df['articles'])
    df['date'] = pd.to_datetime(df['publishedAt'])
    df['date'] = df['date'].dt.floor('4H')

    analyzer = SentimentIntensityAnalyzer()

    # Vader Polarity
    df = pd.concat([df, pd.DataFrame([analyzer.polarity_scores(text) for text in df['title']])], axis=1)

    # TextBlob Subjectivity
    df['subjectivity'] = [TextBlob(text).sentiment.subjectivity for text in df['title']]
    df = df.fillna(0)
    return df['neg'].mean() * 100 + 50, df['pos'].mean() * 100 + 50, df['compound'].mean() * 100 + 50

def get_keyword_sentiment(keyword='nasdaq'):
  end_date = datetime.datetime.now().strftime('%Y-%m-%d')
  start_date = (datetime.datetime.now() - datetime.timedelta(days=3)).strftime('%Y-%m-%d')

  df = api.get_everything(q=keyword, language='en', page_size=100, from_param=start_date, to=end_date)
  df = pd.DataFrame(df['articles'])
  df['date'] = pd.to_datetime(df['publishedAt'])
  df['date'] = df['date'].dt.floor('4H')

  analyzer = SentimentIntensityAnalyzer()

  df = pd.concat([df, pd.DataFrame([analyzer.polarity_scores(text) for text in df['title']])], axis=1)

  df['subjectivity'] = [TextBlob(text).sentiment.subjectivity for text in df['title']]
  df = df.fillna(0)
  return df['neg'].mean() * 100 + 50, df['pos'].mean() * 100 + 50, df['compound'].mean() * 100 + 50


In [ ]:
business_sentiment = get_headline_sentiment()
general_sentiment = get_headline_sentiment('general')
nasdaq_sentiment = get_keyword_sentiment('Nasdaq')
snp500_sentiment = get_keyword_sentiment('S&P 500')
dowjones_sentiment = get_keyword_sentiment('Dow Jones')

neg = (business_sentiment[0] + general_sentiment[0] + nasdaq_sentiment[0] + snp500_sentiment[0] + dowjones_sentiment[0]) / 5
pos = (business_sentiment[1] + general_sentiment[1] + nasdaq_sentiment[1] + snp500_sentiment[1] + dowjones_sentiment[1]) / 5
compound = (business_sentiment[2] + general_sentiment[2] + nasdaq_sentiment[2] + snp500_sentiment[2] + dowjones_sentiment[2]) / 5

# Optional: If you want to convert neg, pos, compound to integers, you can use round() or int()
neg = int(round(neg))
pos = int(round(pos))
compound = int(round(compound))

In [ ]:
data = {'positive': [34, 42, 34, 33, 11, 77, pos],
        'negative': [12, 35, 12, 55, 11, 13, neg],
        'compound': [22, 10, 6, 10, 23, 84, compound]}

df = pd.DataFrame(data, index=['2022/11/11', '2022/11/12', '2022/11/13', '2022/11/14', '2022/11/15', '2022/11/16', '2022/11/17'])
df.index.name = 'date'
df

,positive,negative,compound
date,,,
2022/11/11,34,12,22
2022/11/12,42,35,10
2022/11/13,34,12,6
2022/11/14,33,55,10
2022/11/15,11,11,23
2022/11/16,77,13,84
2022/11/17,59,56,55


In [ ]:
df.to_csv('sentiment_df.csv', encoding="utf-8-sig")
files.download('sentiment_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
# word cloud
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 49.1 MB/s eta 0:00:00


In [2]:
from multiprocessing import Pool
from konlpy.tag import Hannanum
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html

class News:
  def __init__(self):
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_ranking(self, page=1, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    pages = list(range(1,page+1))
    li = []
    for page in pages:
      url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date=' + date + '&page=' + str(page)
      result = requests.get(url, headers = self.headers)
      bs_obj = BeautifulSoup(result.content, "html.parser")

      div = bs_obj.find_all('div', {'class':'hotNewsList'})
      li+=[html.unescape(x['href']) for x in div[0].find_all('a')]
    return li

  def get_article(self, url):
    url = 'https://finance.naver.com/' + url
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    title = ' '.join(bs_obj.find_all('div', {'class': 'article_info'})[0].find_all('h3')[0].get_text().split())
    context = bs_obj.find_all('div', {'class': 'articleCont'})[0]
    try:
      subheading = context.find_all('strong')[0].get_text()
    except:
      subheading = ''
    t = ' '.join(context.get_text().split())
    text = t[:t.find('@')]
    text = text.replace(subheading, '')
    return title, subheading, text

news = News()
news_list = news.get_ranking(5)
with Pool() as p:
    li = p.map(news.get_article, news_list)
li = np.array(li)

dic = {'Title':li[:,0], 'Subheading':li[:,1], 'Text':li[:,2]}
df = pd.DataFrame.from_dict(dic)
df

,Title,Subheading,Text
0,휴가철도 막바지…'백캉스'로 빵지순례 다녀볼까,[주말쇼핑포인트]롯데 개성주악·더현대 뵈르뵈르 아이스크림신세계 가루쌀 빵 팝업…집객...,전통 개성주악 디저트 '세화연' 팝업 이미지(롯데백화점 제공)(서울=뉴스1) 서미선...
1,중국인 단체관광길 열렸지만…과거 영광 되찾기는 ‘글쎄’ [언박싱],유커의 귀환 ②,중국인의 한국행 단체관광이 전면 허용되면서 중국 관광객(유커·遊客)이 속속 입국하고...
2,나흘간 6500억 판 아트페어 또 열린다…미술시장 뛰어드는 유통가,"세계 3대 아트페어 프리즈, 오는 9월 두 번째 서울 행사국내 미술시장, 지난해 1...",지난해 9월 코엑스에서 열린 아트페어 '프리즈 서울'을 찾은 관람객들이 전시장 입장...
3,"디폴트 위기의 中 비구이위안, 홍콩 항셍지수서 제외",,중국 대형 부동산 개발업체 비구이위안(碧桂園·컨트리가든)이 채무불이행(디폴트) 위기...
4,나스닥 4일 연속 하락…휴가시즌 낙관론에 대한 의문 [뉴욕마감],,뉴욕 타임스퀘어 나스닥 마켓플레이스뉴욕증시에서 3대 지수가 이번주 마지막 거래일을 ...
...,...,...,...
118,"""너무 줄였나"" 증권사, 지점 감축 바람… 삼성 3년새 '반토막'",,국내 증권사들이 오프라인 지점을 줄이고 온라인 채널 확대에 나섰다. 사진은 서울 여...
119,보험권 '50년 만기 주담대' 시대 열리나… 삼성화재·생명도 합류,,대형 보험사들이 50년 만기 주담대를 속속 출시하며 보험권에 전반적으로 확산될지 이...
120,'헝다 위기감 고조' 中 펀드도 울상… 한달새 4000억 빠졌다,,중국 전에 있는 중국 에버그란데 그룹 본사./사진=로이터 국내 중국 주식형 펀드에서...
121,"7000만원대 GV80, 카드 할부로 살까… 신한 vs KB국민, 車할부 키운다",,사진=이미지투데이 신차 할부금융 시장에 카드사들이 드라이브를 걸고 있다. 카드사들은...


In [51]:
import random
import re

hannanum = Hannanum()

def clean_title(title):
    cleaned_title = title.replace(',', ' ') \
                         .replace('(', ' ') \
                         .replace(')', ' ') \
                         .replace('…', ' ') \
                         .replace('?', ' ') \
                         .replace('.', ' ') \
                         .replace("'", ' ') \
                         .replace('"', ' ') \
                         .replace('”', ' ') \
                         .replace('“', ' ') \
                         .replace('‘', ' ') \
                         .replace('’', ' ') \
                         .replace('·', ' ') \
                         .replace('[', ' ') \
                         .replace(']', ' ') \
                         .replace('↓', ' ') \
                         .replace('→', ' ') \
                         .replace('↑', ' ')
    return cleaned_title

def rank_words(words_list):
  word_ranking = {}
  for word in words_list:
      if word in word_ranking:
          word_ranking[word] += 1
      else:
          word_ranking[word] = 1
  sorted_word_ranking = dict(sorted(word_ranking.items(), key=lambda item: item[1], reverse=True))
  return sorted_word_ranking

def standardize_values(dictionary):
    max_value = max(dictionary.values())
    min_value = min(dictionary.values())

    for key in dictionary:
        value = dictionary[key]
        standardized_value = 10 + (value - min_value) * (90 / (max_value - min_value))
        dictionary[key] = round(standardized_value)

    return dictionary

def custom_standardize_values(dictionary, new_min, new_max, noise_min, noise_max):
    max_value = max(dictionary.values())
    min_value = min(dictionary.values())

    standardized_dict = {}
    for key, value in dictionary.items():
        # Linear transformation to scale the values
        standardized_value = new_min + (value - min_value) * (new_max - new_min) / (max_value - min_value)

        # Add random noise
        noise = random.uniform(noise_min, noise_max)
        standardized_value_with_noise = standardized_value + noise

        standardized_dict[key] = round(standardized_value_with_noise, 0)

    return standardized_dict

def extract_nouns(text, custom_nouns_list):
    words = re.findall(r'\b\w+\b', text)

    filtered_nouns = {}
    for word in words:
        if word in custom_nouns_list:
            filtered_nouns[word] = filtered_nouns.get(word, 0) + 1

    return filtered_nouns

In [4]:
df['Cleaned_Title'] = df['Title'].apply(clean_title)
df['Filtered_Pos'] = [hannanum.nouns(x) for x in df['Cleaned_Title']]
words = [word for sublist in df['Filtered_Pos'] for word in sublist]
word_frequency_dict = rank_words(words)
result = standardize_values(word_frequency_dict)

In [40]:
stop_words = ['의', '등', '때', '오늘', '이곳', '저', '월', '년', '일', '로', '어디', '데', '특징주', '장중', '전', '주', '위', '한', '수',' 속', '전', '추', '속', '내', '비트코']
data_dict = {key: value for key, value in result.items() if key not in stop_words}
data_dict = {key: value for key, value in data_dict.items() if not str(key).isdigit()}

In [ ]:
#설정 종목명

title_as_sentence = ' '.join(df['Cleaned_Title'].values)
custom_nouns = stocklist.df['종목명'].values.tolist() + ['비트코인', '이더리움']
filtered_nouns = extract_nouns(title_as_sentence, custom_nouns)
ticker_dict = custom_standardize_values(filtered_nouns, 40, 80, 5, 15)

In [54]:
data_dict.update(ticker_dict)

In [62]:
t = pd.DataFrame(data_dict, index=['value']).T.sort_values(by='value', ascending=False).head(50)

In [63]:
t

,value
中,100.0
비트코인,87.0
뉴욕증시,68.0
셀트리온,65.0
부동산,61.0
SK텔레콤,55.0
이마트,55.0
삼성화재,55.0
레인보우로보틱스,54.0
네패스아크,54.0


In [ ]:
from google.colab import files

t.to_csv('words_df.csv', header=False, encoding="utf-8-sig")
files.download('words_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>